# ISMAILA LE GOAT

In [1]:
import pandas as pd
import numpy as np


In [5]:
#load data 
df=pd.read_csv("../code/achat_prod_fournisseur_stock.csv") #upload from code file

In [6]:
df.head()

,id_achat,date_achat,id_produit,quantité,id_fournisseur,prix_unitaire,délai_livraison_jours,montant_total,mois,année,...,marque,prix,stock_minimum,nom_fournisseur,ville,pays,fiabilité,délai_moyen_jours,niveau_stock,entrepot
0,PO000001,2024-03-22,P00627,150,S029,129.62,14,19443.0,3,2024,...,Atelier Minéral,39.95,18,Fontaine Lopes S.A.,Helsinki,Finlande,0.54,6,289,Paris
1,PO000002,2024-06-12,P00469,50,S028,110.81,8,5540.5,6,2024,...,Reflet d'or,25.58,18,Delattre Blot S.A.,Berlin,Allemagne,0.74,8,80,Paris
2,PO000003,2024-01-19,P00849,75,S001,140.58,8,10543.5,1,2024,...,Ãveil Ãlégant,62.78,83,Lecomte et Fils,Zurich,Suisse,0.84,10,54,Marseille
3,PO000004,2024-11-20,P00484,500,S028,120.77,9,60385.0,11,2024,...,Reflet d'or,259.47,39,Delattre Blot S.A.,Berlin,Allemagne,0.74,8,249,Marseille
4,PO000005,2a024-06-07,P00196,50,S025,256.05,13,12802.5,6,2024,...,Luxe eclat,353.45,33,Techer SA,Varsovie,Pologne,0.92,12,119,Paris


## Hi

In [ ]:

# 0. Imports et configuration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score, roc_curve, precision_score, recall_score, f1_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from xgboost import XGBRegressor, XGBClassifier

# 1. Chargement et préparation des données
# Lecture depuis les CSV extraits
df_cmd = pd.read_csv('/mnt/data/commandes.csv', parse_dates=['date_commande'])
df_exp = pd.read_csv('/mnt/data/expeditions.csv', parse_dates=['date_expedition','date_livraison'])
df_ret = pd.read_csv('/mnt/data/retours.csv', parse_dates=['date_retour'])
df_stock = pd.read_csv('/mnt/data/stocks.csv', parse_dates=['date_stock'])
df_achats = pd.read_csv('/mnt/data/achats.csv', parse_dates=['date_achat'])
df_fourn = pd.read_csv('/mnt/data/fournisseurs.csv')
df_prod = pd.read_csv('/mnt/data/produits.csv')

# Harmonisation texte
for df in [df_cmd, df_exp, df_ret, df_stock, df_achats, df_fourn, df_prod]:
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].str.strip().str.lower()

# 2. Calcul des KPI
# 2.1 OTIF
df_exp['on_time'] = (df_exp['date_livraison'] <= df_exp['date_attendue']).astype(int)
df_exp['full']    = (df_exp['quant_livree'] >= df_exp['quant_attendue']).astype(int)
df_exp['otif_flag'] = df_exp['on_time'] * df_exp['full']
otif_pct = df_exp['otif_flag'].mean() * 100

# 2.2 Taux de retour
ret_tot   = df_ret.groupby('id_commande')['quantite_retournee'].sum()
deliv_tot = df_exp.groupby('id_commande')['quant_livree'].sum()
ret = ret_tot.to_frame('quantite_retournee').join(deliv_tot.to_frame('quant_livree'), how='inner')
retour_pct = (ret['quantite_retournee'].sum() / ret['quant_livree'].sum()) * 100

# 2.3 Délai moyen expédition
df_exp['lead_time'] = (df_exp['date_livraison'] - df_exp['date_expedition']).dt.days
delai_moyen = df_exp['lead_time'].mean()

# 2.4 Rotation des stocks
# consommation annuelle (quantité expédiée) et stock moyen
year = df_stock['date_stock'].dt.year.max()
consommation = df_exp[df_exp['date_expedition'].dt.year==year]['quant_livree'].sum()
stock_deb = df_stock[df_stock['date_stock']==pd.to_datetime(f"{year}-01-01")]['niveau_stock'].mean()
stock_fin = df_stock[df_stock['date_stock']==pd.to_datetime(f"{year}-12-31")]['niveau_stock'].mean()
rotation_stock = consommation / ((stock_deb + stock_fin)/2)

# 2.5 Coût logistique unitaire
cout_unitaire = df_exp['cout_transport'].sum() / df_exp['quant_livree'].sum()

# 2.6 Fill Rate
fill_rate = df_exp['quant_livree'].sum() / df_exp['quant_attendue'].sum() * 100

# Regroup KPI
glob_kpis = pd.Series({
    'OTIF_%': otif_pct,
    'Taux_retour_%': retour_pct,
    'Delai_moyen_jours': delai_moyen,
    'Rotation_stock': rotation_stock,
    'Cout_unitaire': cout_unitaire,
    'Fill_rate_%': fill_rate
})

# 3. EDA rapide
glob_kpis.plot(kind='bar', title='KPI Supply Chain'); plt.tight_layout(); plt.show()

# 4. Feature Engineering pour ML
# Sur base commandes
df_ml = df_cmd.merge(ret[['quantite_retournee']], on='id_commande', how='left').fillna(0)
# Variables temporelles
df_ml['mois'] = df_ml['date_commande'].dt.month
df_ml['jour_semaine'] = df_ml['date_commande'].dt.dayofweek
# Rolling commandes mensuelles :
monthly = df_ml.set_index('date_commande')['quantite_retournee'].resample('M').sum()
df_ml = df_ml.merge(monthly.shift(1).rename('roll_1m'),
                    left_on=df_ml['date_commande'].dt.to_period('M').dt.to_timestamp(),
                    right_index=True, how='left')

# 5. Forecasting de la demande
# Préparation séries temporelles journalière
df_ts = df_cmd.set_index('date_commande').groupby('id_produit')['quantite'].resample('D').sum().fillna(0)
results = {}
prods = df_ts.index.get_level_values(0).unique()[:3]
for pid in prods:
    ts = df_ts.loc[pid]
    train, test = ts[:-30], ts[-30:]
    # SARIMAX
    model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,7)).fit(disp=False)
    sar_pred = model.predict(start=test.index[0], end=test.index[-1])
    mae_s = mean_absolute_error(test, sar_pred)
    # Prophet
    df_prop = train.reset_index().rename(columns={'date_commande':'ds','quantite':'y'})
    m = Prophet(); m.fit(df_prop)
    future = m.make_future_dataframe(periods=30, freq='D')
    pr = m.predict(future); pr_pred = pr.set_index('ds')['yhat'].loc[test.index]
    mae_p = mean_absolute_error(test, pr_pred)
    # RF
    lags = pd.concat([train.shift(lag) for lag in [1,7,14]], axis=1)
    lags.columns = ['lag1','lag7','lag14']
    lags = lags.dropna()
    X = lags.values; y = train.loc[lags.index].values
    rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X,y)
    X_test = np.vstack([test.shift(lag).loc[lags.index].values for lag in [1,7,14]]).T
    mae_rf = mean_absolute_error(test.loc[lags.index], rf.predict(X_test))
    results[pid] = {'SARIMAX_MAE':mae_s,'Prophet_MAE':mae_p,'RF_MAE':mae_rf}

# Affichage comparatif
res_df = pd.DataFrame(results).T
res_df.plot(kind='bar', title='MAE comparatif Forecast'); plt.tight_layout(); plt.show()

# 6. Clustering de produits & fournisseurs
# Produits
df_p = df_achats.groupby('id_produit').agg({'quant_achetee':'sum','prix_unitaire':'mean'}).reset_index()
k_p = KMeans(n_clusters=4, random_state=0).fit(df_p[['quant_achetee','prix_unitaire']])
df_p['cluster'] = k_p.labels_
# Fournisseurs
df_f = df_achats.merge(df_fourn[['id_fournisseur','delai_moyen_jours']],on='id_fournisseur')
df_f = df_f.groupby('id_fournisseur').agg({'quant_achetee':'sum','delai_moyen_jours':'mean'}).reset_index()
k_f = DBSCAN(eps=10, min_samples=2).fit(df_f[['quant_achetee','delai_moyen_jours']])
df_f['db_cluster'] = k_f.labels_

# PCA pour visualiser clusters fabricants
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
comp = pca.fit_transform(df_f[['quant_achetee','delai_moyen_jours']])
df_f['pca1'], df_f['pca2'] = comp[:,0], comp[:,1]
plt.figure()
plt.scatter(df_f['pca1'], df_f['pca2'], c=df_f['db_cluster'], cmap='tab10')
plt.title('Clusters Fournisseurs (DBSCAN)')
plt.show()

# 7. Modèles de classification des retards
df_delay = df_exp.copy()
df_delay['late'] = (df_delay['lead_time']>3).astype(int)
X = pd.get_dummies(df_delay[['quant_livree','cout_transport','entrepôt']], drop_first=True)
y = df_delay['late']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=42)
def eval_clf(m):
    m.fit(X_train,y_train)
    pred = m.predict(X_test); prob = m.predict_proba(X_test)[:,1]
    print(m.__class__.__name__)
    print(f"Precision: {precision_score(y_test,pred):.2f}, Recall: {recall_score(y_test,pred):.2f}, F1: {f1_score(y_test,pred):.2f}, AUC: {roc_auc_score(y_test,prob):.2f}")
for model in [RandomForestClassifier(n_estimators=100), XGBClassifier(use_label_encoder=False, eval_metric='logloss')]:
    eval_clf(model)

# Courbe ROC pour RF
rf_clf = RandomForestClassifier(n_estimators=100).fit(X_train,y_train)
fpr,tpr,_ = roc_curve(y_test, rf_clf.predict_proba(X_test)[:,1])
plt.figure(); plt.plot(fpr,tpr); plt.title('ROC RF'); plt.show()

# 8. Hyperparamétrage example pour RF rég.
param_dist = {'n_estimators':[50,100,200],'max_depth':[None,5,10]}
rs = RandomizedSearchCV(RandomForestRegressor(), param_dist, n_iter=5, cv=3, scoring='neg_mean_absolute_error',random_state=42)
rs.fit(df_p[['quant_achetee']], df_p['quant_achetee'])
print('Best Params Reg:', rs.best_params_)


## Remarque:
- Remplacement des connexions SQL par lecture CSV.
- Fusion des tables selon votre schéma de projet.
- Intégration forecasting (SARIMAX, Prophet, RF), clustering (KMeans, DBSCAN), classification retards.
- Visualisations matplotlib intégrées.
- Exemples d’hyperparamétrage avec RandomizedSearchCV.
